Retail Customer Segmentation with RFM Model
This project analyzes retail customer transactions data and classifies customers into groups by applying RFM model and quantile scoring method. The objective is to help the business gain insights and run effective marketing campaigns.

RFM model is a marketing concept for segmenting customers based on their buying behaviors in 3 areas: Recency, Frequency, and Monetary.
We are going through five steps in this project:
1. Load Data
2. Cleaning Data
3. Manipulating Data
4. Segmentation
5. Conclusion

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Read the dataset
df= pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTywFpUKSOSvi0xZosRzpXu1gxjt-dpXLuJ2Yo_FV_MFuJz3eukbvjx603dKxjW5DJdpUDPnxjIjPd9/pub?output=csv', parse_dates=['InvoiceDate'])
df.head(10)


ModuleNotFoundError: No module named 'pandas'

In [17]:
#Get a summary of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


The dataset contains 541,909 rows and 8 columns. InvoiceDate has already been converted to datetime type when reading the file. The rests of them are in acceptable data types.


In [18]:
#How much values are in each field?
df[df.columns[1:]].nunique()

StockCode       4070
Description     4211
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

Counting unique values in each column shows that there are more than 4,300 customers from 38 countries having purchased over 4,000 different products in the time period.
There are about 4000 products sold as well.
But we need to be sure of the accurate of the data so we are going to see if there are missing data.

In [19]:
#Check for missing values
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

There are 135,080 records (25% of total) with no customer ID and around 1,400 without description. Only CustomerID is important for the analysis so I will drop all rows with null in customerID. 

In [ ]:
#View the records with missing CustomerID
df[df['CustomerID'].isnull()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom


In [29]:
#Drop rows with missing CustomerID
df.dropna(subset = 'CustomerID', inplace = True) 
df.shape

(406829, 8)

In [27]:
#Check for duplicates
df.duplicated().sum()

AttributeError: 'NoneType' object has no attribute 'duplicated'

There are 5,225 duplicates in this dataset with duplicate values across all columns.
The nest step is drop all the duplicate values.

In [30]:
#Drop rows with missing CustomerID
df.drop_duplicates(keep='last', inplace=True)

df.shape

(401604, 8)

In [31]:
# Check data in numeric fields
df[['Quantity', 'UnitPrice']].agg({'Quantity':['min', 'mean', 'max'], 'UnitPrice':['min', 'mean', 'max']})

,Quantity,UnitPrice
min,-80995.000000,0.000000
mean,12.183273,3.474064
max,80995.000000,38970.000000


In [32]:
# Check for negative values in Quantity and UnitPrice
print(df[df['Quantity'] <= 0].shape[0])
print(df[df['UnitPrice'] == 0].shape[0])
print(df[(df['Quantity'] <= 0) | (df['UnitPrice'] == 0)].shape[0])

8872
40
8912


There are 8,872 records with negative values in Quantity and 40 records with zero in UnitPrice (8,912 with either condition).

In [ ]:
# Exclude rows with negative or zero in both fields
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

df.shape

3. DATA MANIPULATION

These are new columns needed for RFM Analysis:

#SumTotal: quantity times unit price (will be used as input for Monetary).
#Recency: number of days from “current” date.
#Frequency: count of unique invoices.
#Monetary: sum of dollar values from all purchases

In [33]:
#Create SumTotal column
df['SumTotal'] = df['Quantity'] * df['UnitPrice']
df[['Quantity', 'UnitPrice', 'SumTotal']].head()

,Quantity,UnitPrice,SumTotal
0,6,2.55,15.30
1,6,3.39,20.34
2,8,2.75,22.00
3,6,3.39,20.34
4,6,3.39,20.34


In [ ]:
#Find the average customer spend per time
basket_size=df.groupby('InvoiceNo')['SumTotal'].sum()
print("Average customer spend per time is:  $",round(basket_size.mean()))


Average customer spend per time is:  $ 373


In [ ]:
#Recency column
recency_df=(df['InvoiceDate'].max()-df.groupby('CustomerID')['InvoiceDate'].max()).dt.days
#Frecuency 
frequency_df=df.groupby('CustomerID')['InvoiceNo'].nunique()
#Monetary
monetary_df=df.groupby('CustomerID')['SumTotal'].sum()
#Merge RFM columns into a new dataframe
rfm_df=pd.DataFrame({'Recency':recency_df, 'Frequency':frequency_df, 'Monetary':monetary_df})
rfm_df.head()